In [2]:
pip install pysam

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pysam as pys

In [4]:
# Take the first read from the AlignmentFile
AlignmentFile = pys.AlignmentFile('/sbgenomics/workspace/merged-tumor.bam','rb')

for read in AlignmentFile:
    AlignedSegment = read  
    print(read)
    break
    

C0HVYACXX120402:7:1207:5722:57044	1187	20	9483248	27	76M	20	9483381	76	TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG	array('B', [28, 28, 27, 29, 31, 30, 31, 31, 29, 31, 35, 30, 29, 31, 34, 30, 29, 23, 41, 32, 20, 30, 29, 34, 34, 29, 30, 31, 30, 30, 30, 33, 33, 26, 39, 12, 25, 19, 32, 30, 35, 28, 35, 33, 23, 33, 35, 36, 30, 38, 33, 41, 34, 35, 31, 33, 23, 30, 30, 36, 27, 32, 29, 34, 35, 41, 33, 31, 33, 29, 32, 32, 31, 31, 31, 34])	[('XA', 'GL000217.1,-110754,76M,1;'), ('BD', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('MD', '76'), ('RG', '1'), ('BI', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NM', 0), ('MQ', 27), ('AS', 76), ('XS', 71)]


In [5]:
# Inspect the fields in the AlignedSegment

AlignedSegment.to_dict()

{'name': 'C0HVYACXX120402:7:1207:5722:57044',
 'flag': '1187',
 'ref_name': '21',
 'ref_pos': '9483249',
 'map_quality': '27',
 'cigar': '76M',
 'next_ref_name': '=',
 'next_ref_pos': '9483382',
 'length': '209',
 'seq': 'TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG',
 'qual': '==<>@?@@>@D?>@C?>8JA5?>CC>?@???BB;H-:4A?D=DB8BDE?GBJCD@B8??E<A>CDJB@B>AA@@@C',
 'tags': ['XA:Z:GL000217.1,-110754,76M,1;',
  'BD:Z:NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN',
  'MD:Z:76',
  'RG:Z:1',
  'BI:Z:NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN',
  'NM:i:0',
  'MQ:i:27',
  'AS:i:76',
  'XS:i:71']}

In [6]:
# Inspect the flag field

# we should use webpage on this link: https://broadinstitute.github.io/picard/explain-flags.html
# in order to decode value of flag.

print("AlignedSegment.flag = ",AlignedSegment.flag)
print("")
print("///////////////////////////////////////////////////////////////////////////////////////////////////")
print("//                                Decoding SAM flag (1187)                                       //")
print("///////////////////////////////////////////////////////////////////////////////////////////////////")
print("")
print("first in pair:                                             second in pair: \n")
print("read paired (0x1)                                          read paired (0x1)")
print("read mapped in proper pair (0x2)                           read mapped in proper pair (0x2)")
print("mate reverse strand (0x20)                                 read reverse strand (0x10)")
print("second in pair (0x80)                                      first in pair (0x40)")
print("read is PCR or optical duplicate (0x400)                   read is PCR or optical duplicate (0x400)")

AlignedSegment.flag =  1187

///////////////////////////////////////////////////////////////////////////////////////////////////
//                                Decoding SAM flag (1187)                                       //
///////////////////////////////////////////////////////////////////////////////////////////////////

first in pair:                                             second in pair: 

read paired (0x1)                                          read paired (0x1)
read mapped in proper pair (0x2)                           read mapped in proper pair (0x2)
mate reverse strand (0x20)                                 read reverse strand (0x10)
second in pair (0x80)                                      first in pair (0x40)
read is PCR or optical duplicate (0x400)                   read is PCR or optical duplicate (0x400)


In [7]:
# How many unmapped reads are in the file ?
# We need to convert decimal to binary and then to see if bit 2 has value one ==> read is unmaped

# Axilary function

def DecimalToBinary(num):
    binary_array = [] 
    while num >= 1:
        binary_array.append(num % 2)
        num = num // 2
    
    return binary_array

    
array = DecimalToBinary(163)
array.reverse()
print(array)



[1, 0, 1, 0, 0, 0, 1, 1]


In [8]:
# How many unmapped reads are in the file ?
unmapped_reads = 0
for read in pys.AlignmentFile('/sbgenomics/workspace/merged-tumor.bam','rb'):
    array = DecimalToBinary(read.flag)
    if len(array) >= 3 and array[2] == 1:
        unmapped_reads = unmapped_reads + 1
    
print("unmapped reads number is ", unmapped_reads)

unmapped reads number is  17765


In [9]:
# Total number of reads ?
def total_reads_num():
    total_reads = 0
    for read in pys.AlignmentFile('/sbgenomics/workspace/merged-tumor.bam','rb'):
        total_reads = total_reads + 1
        
    return total_reads

print("total number of reads is = ", total_reads_num())

total number of reads is =  2921629


In [10]:
# Number of reads with mapping quality zero
quality_zero = 0
for read in pys.AlignmentFile('/sbgenomics/workspace/merged-tumor.bam','rb'):
    if read.mapping_quality == 0 :
        quality_zero = quality_zero + 1

print("Number of reads with zero quality is = ", quality_zero)

Number of reads with zero quality is =  126628


In [11]:
# Average mapping quality for all the reads ?
average_quality = 0
for read in pys.AlignmentFile('/sbgenomics/workspace/merged-tumor.bam','rb'):
    average_quality = average_quality + read.mapping_quality

average_quality = average_quality / total_reads_num()
print("Average mapping quality for all the reads is = ", average_quality)

Average mapping quality for all the reads is =  55.91379158681681


In [12]:
# Average mapping quality if reads with 0 mapp quality are filtered out ?
reads_with_zero_quality = 0
average_quality = 0
for read in pys.AlignmentFile('/sbgenomics/workspace/merged-tumor.bam','rb'):
    if (read.mapping_quality > 0):
        average_quality = average_quality + read.mapping_quality
    else:
        reads_with_zero_quality = reads_with_zero_quality + 1
    

average_quality = average_quality / (total_reads_num() - reads_with_zero_quality )

print("Average mapping quality for all the reads when reads with 0 mapp quality are filtered out  is = ", average_quality)

Average mapping quality for all the reads when reads with 0 mapp quality are filtered out  is =  58.446975510921106
